# 📦 日本貿易統計データ処理ノートブック

## 🔹 1. ライブラリのインポート



In [1]:
from library.HScode_scrape import (generate_customs_urls,fetch_and_concat_data,validate_and_log_hs_dataframe)
from library.get_export_data_HSitem import TradeDataPipeline
import pandas as pd

---

## 🔹 2. HSコードマスタの取得・保存

### 対象年・月・番号範囲の指定

In [ ]:
year, month = 2010, 1
num_range = range(1, 98)

### データ取得 & 検証 & 保存

In [10]:
# HSコードのスクレイピング実行
urls = generate_customs_urls(year, month, num_range)
df = fetch_and_concat_data(urls)
log_df = validate_and_log_hs_dataframe(df, year)

# CSVで保存
df.to_csv(f'./reference_master/HS_master/HSコードマスタ_{year:04d}.csv', encoding='utf-8', index=False)

✅ 検査ログを保存しました: ./reference_master/HS_master/log\2025-04-22_20-31-05_hs_check_log_2010.csv


## 🔹 3. e-Stat API を用いた貿易データの取得と保存

### パラメータの設定

In [2]:
# 分類種別、対象年月、APIキーの指定
syurui = 'HS'  # 品目別
year, month = 2024, '01'
api_key = "cd892ddafd38c70af31204e5e2927a3573fa942e"

### マスタの読み込みとフィルタリング

In [3]:
# 貿易統計マスタと対象データの抽出
trade_counter_df = pd.read_excel('./reference_master/counter/貿易統計_対応表.xlsx', dtype=str)
hs_counter_df = trade_counter_df[(trade_counter_df['分類'] == syurui) & (trade_counter_df['year'].astype(int) == year)].reset_index(drop=True)

# 国コードマスタの読み込み
nation_df = pd.read_excel('./reference_master/area/nation.xlsx')

### パイプライン実行

In [4]:
pipeline = TradeDataPipeline(hs_counter_df, trade_counter_df, nation_df, year, month, api_key)
pipeline.run()

✅ 初期化完了: 取得開始
▶️ 分類：HS 2024年 01月
📁 保存完了: ./Output/HS_item/2024_01_国別_税関別_HS品目別.csv
⏱️ 実行完了: 1分 46秒


### 一括実行

In [ ]:
api_key = "cd892ddafd38c70af31204e5e2927a3573fa942e"
syurui = 'HS'  # 品目別
for year in range (2010, 2024):
    for month in ['01','02','03','04','05','06','07','08','09','10','11','12']:
        # 分類種別、対象年月、APIキーの指定
        syurui = 'HS'
        # 貿易統計マスタと対象データの抽出
        trade_counter_df = pd.read_excel('./reference_master/counter/貿易統計_対応表.xlsx', dtype=str)
        hs_counter_df = trade_counter_df[(trade_counter_df['分類'] == syurui) & (trade_counter_df['year'].astype(int) == year)].reset_index(drop=True)

        # 国コードマスタの読み込み
        nation_df = pd.read_excel('./reference_master/area/nation.xlsx')
        pipeline = TradeDataPipeline(hs_counter_df, trade_counter_df, nation_df, year, month, api_key)
        pipeline.run()

✅ 初期化完了: 取得開始
▶️ 分類：HS 2010年 01月
📁 保存完了: ./Output/HS_item/2024_01_税関別_20250423.csv
⏱️ 実行完了: 2分 1秒
✅ 初期化完了: 取得開始
▶️ 分類：HS 2010年 02月


KeyboardInterrupt: 

### ✅ 出力物

- `./reference_master/HS_master/HSコードマスタ_{year}.csv` : HSコードマスタデータ
- `./Output/HS_item/` 以下 : 月次の貿易データ（数量・金額）CSV出力
- ログファイル : 不正なデータ行の記録

---

### 📝 メモ
- `.gitignore` に出力ファイルパスを追加しておくと、誤ってリポジトリに大容量CSVを含めることを防げます。
- Jupyter Notebook 上では各セルごとに実行できるため、エラー発生時も柔軟にデバッグ可能です。
